In [1]:
import itertools
import math
from tkinter import *
from tkinter import filedialog

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [17]:
def adp_filt(currdf: object, pose):
    lIndex = []
    xIndex = []
    yIndex = []
    currdf = np.array(currdf[1:])
    for header in pose:
        if currdf[0][header + 1] == "likelihood":
            lIndex.append(header)
        elif currdf[0][header + 1] == "x":
            xIndex.append(header)
        elif currdf[0][header + 1] == "y":
            yIndex.append(header)
    curr_df1 = currdf[:, 1:]
    datax = curr_df1[1:, np.array(xIndex)]
    datay = curr_df1[1:, np.array(yIndex)]
    data_lh = curr_df1[1:, np.array(lIndex)]
    currdf_filt = np.zeros((datax.shape[0], (datax.shape[1]) * 2))
    perc_rect = []
    for i in range(data_lh.shape[1]):
        perc_rect.append(0)
    for x in tqdm(range(data_lh.shape[1])):
        a, b = np.histogram(data_lh[1:, x].astype(np.float64))
        rise_a = np.where(np.diff(a) >= 0)
        if rise_a[0][0] > 1:
            llh = b[rise_a[0][0]]
        else:
            llh = b[rise_a[0][1]]
        # llh=0
        data_lh_float = data_lh[:, x].astype(np.float64)
        perc_rect[x] = np.sum(data_lh_float < llh) / data_lh.shape[0]
        currdf_filt[0, (2 * x):(2 * x + 2)] = np.hstack([datax[0, x], datay[0, x]])
        for i in range(1, data_lh.shape[0]):
            if data_lh_float[i] < llh:
                currdf_filt[i, (2 * x):(2 * x + 2)] = currdf_filt[i - 1, (2 * x):(2 * x + 2)]
            else:
                currdf_filt[i, (2 * x):(2 * x + 2)] = np.hstack([datax[i, x], datay[i, x]])
    currdf_filt = np.array(currdf_filt)
    currdf_filt = currdf_filt.astype(np.float64)
    return currdf_filt, perc_rect

def boxcar_center(a, n):
    a1 = pd.Series(a)
    moving_avg = np.array(a1.rolling(window=n, min_periods=1, center=True).mean())

    return moving_avg


def feature_extraction(processed_input_data, framerate = 30):
    window = np.int64(np.round(0.05 / (1 / framerate)) * 2 - 1)
    f = []
    for n in range(len(processed_input_data)):
        data_n_len = len(processed_input_data[n])
        dxy_list = []
        disp_list = []
        for r in range(data_n_len):
            if r < data_n_len - 1:
                disp = []
                for c in range(0, processed_input_data[n].shape[1], 2):
                    disp.append(
                        np.linalg.norm(processed_input_data[n][r + 1, c:c + 2] -
                                       processed_input_data[n][r, c:c + 2]))
                disp_list.append(disp)
            dxy = []
            for i, j in itertools.combinations(range(0, processed_input_data[n].shape[1], 2), 2):
                dxy.append(processed_input_data[n][r, i:i + 2] -
                           processed_input_data[n][r, j:j + 2])
            dxy_list.append(dxy)
        disp_r = np.array(disp_list)
        dxy_r = np.array(dxy_list)
        disp_boxcar = []
        dxy_eu = np.zeros([data_n_len, dxy_r.shape[1]])
        ang = np.zeros([data_n_len - 1, dxy_r.shape[1]])
        dxy_boxcar = []
        ang_boxcar = []
        for l in range(disp_r.shape[1]):
            disp_boxcar.append(boxcar_center(disp_r[:, l], window))
            # disp_boxcar.append(disp_r[:, l])
        for k in range(dxy_r.shape[1]):
            for kk in range(data_n_len):
                dxy_eu[kk, k] = np.linalg.norm(dxy_r[kk, k, :])
                if kk < data_n_len - 1:
                    b_3d = np.hstack([dxy_r[kk + 1, k, :], 0])
                    a_3d = np.hstack([dxy_r[kk, k, :], 0])
                    c = np.cross(b_3d, a_3d)
                    ang[kk, k] = np.dot(np.dot(np.sign(c[2]), 180) / np.pi,
                                        math.atan2(np.linalg.norm(c),
                                                   np.dot(dxy_r[kk, k, :], dxy_r[kk + 1, k, :])))
            dxy_boxcar.append(boxcar_center(dxy_eu[:, k], window))
            ang_boxcar.append(boxcar_center(ang[:, k], window))
            # dxy_boxcar.append(dxy_eu[:, k])
            # ang_boxcar.append(ang[:, k])
        disp_feat = np.array(disp_boxcar)
        dxy_feat = np.array(dxy_boxcar)
        ang_feat = np.array(ang_boxcar)
        f.append(np.vstack((dxy_feat[:, 1:], ang_feat, disp_feat)))

    for m in range(0, len(f)):
        f_integrated = np.zeros(len(processed_input_data[m]))
        for k in range(round(framerate / 10), len(f[m][0]), round(framerate / 10)):
            if k > round(framerate / 10):
                f_integrated = np.concatenate(
                    (f_integrated.reshape(f_integrated.shape[0], f_integrated.shape[1]),
                     np.hstack((np.mean((f[m][0:dxy_feat.shape[0],
                                         range(k - round(framerate / 10), k)]), axis=1),
                                np.sum((f[m][dxy_feat.shape[0]:f[m].shape[0],
                                        range(k - round(framerate / 10), k)]), axis=1)
                                )).reshape(len(f[0]), 1)), axis=1
                )
            else:
                f_integrated = np.hstack(
                    (np.mean((f[m][0:dxy_feat.shape[0], range(k - round(framerate / 10), k)]), axis=1),
                     np.sum((f[m][dxy_feat.shape[0]:f[m].shape[0],
                             range(k - round(framerate / 10), k)]), axis=1))).reshape(len(f[0]), 1)
        if m > 0:
            features = np.concatenate((features, f_integrated), axis=1)
        else:
            features = f_integrated
    return f_integrated

In [14]:
infile = '../../../raw_data/VideoOutputCamera 0-10072023-121830DLC_resnet50_bottomup_clearSep21shuffle1_1030000.csv'
df = pd.read_csv(infile)

/var/folders/gp/nsk60k_n2cz7kk79_pv33d8r0000gn/T/ipykernel_92484/2140753222.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(infile)


In [15]:
df

,scorer,DLC_resnet50_bottomup_clearSep21shuffle1_1030000,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.1,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.2,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.3,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.4,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.5,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.6,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.7,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.8,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.9,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.10,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.11,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.12,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.13,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.14,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.15,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.16,DLC_resnet50_bottomup_clearSep21shuffle1_1030000.17
0,bodyparts,snout,snout,snout,right-forepaw,right-forepaw,right-forepaw,left-forepaw,left-forepaw,left-forepaw,right-hindpaw,right-hindpaw,right-hindpaw,left-hindpaw,left-hindpaw,left-hindpaw,tail-base,tail-base,tail-base
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
2,0,317.0372619628906,173.7179718017578,0.9997759461402893,308.0582275390625,173.99876403808594,0.99847811460495,312.92401123046875,182.60382080078125,0.9999876022338867,265.67236328125,165.05502319335938,0.9999997615814209,279.63037109375,201.38417053222656,0.9999995231628418,241.60708618164062,194.1881561279297,1.0
3,1,317.9444274902344,174.32022094726562,0.9996614456176758,307.7373352050781,174.08026123046875,0.9998396635055542,312.8106994628906,183.2034912109375,0.9999536275863647,265.2131652832031,164.99803161621094,0.9999996423721313,280.026123046875,201.59408569335938,0.999998927116394,240.9491424560547,194.2281036376953,1.0
4,2,314.99444580078125,174.3306884765625,0.999640703201294,311.2739562988281,174.5463104248047,0.9460304379463196,314.2292785644531,182.42726135253906,0.7720528244972229,265.007568359375,165.19085693359375,0.9999997615814209,280.1241760253906,201.64157104492188,0.9999980926513672,241.3518829345703,194.15151977539062,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117547,117545,261.434753,266.454956,0.999997,289.730774,238.415955,0.981241,266.281464,227.789398,0.999995,293.732666,163.694641,1.0,266.425446,207.677856,0.999994,261.62793,167.737198,1.0
117548,117546,261.166595,271.104156,0.999997,290.269318,242.623337,0.999231,266.802704,227.926666,0.999996,293.55777,164.218201,1.0,266.642365,207.600754,0.999994,261.778625,168.508423,1.0
117549,117547,260.063477,272.195221,1.0,290.524841,243.366608,0.999915,267.204865,228.991928,0.999995,292.55658,164.58696,1.0,266.582275,207.815948,0.999996,260.77417,168.080841,1.0
117550,117548,259.946747,272.452118,1.0,279.209076,238.821594,0.643459,266.941132,228.735565,0.999997,292.899597,164.403442,1.0,266.559174,208.019073,0.999988,260.560486,168.03949,1.0


In [16]:
with open('../../../model/clf.pkl', 'rb') as fr:
    clf = pickle.load(fr)

In [18]:
# filter likelihood (smoothing)
csv_array_filtered, perc_filtered = adp_filt(df, np.arange(18))
f_integrated = feature_extraction([csv_array_filtered.copy()])

100%|███████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.95it/s]


In [22]:
X = f_integrated.T.copy()
predictions = clf.predict(X)

In [28]:
# 100ms, so 3000 * 0.1s is actually 300s
class_of_interest = 2 # locomotion
behavior_names = ['left', 'right', 'locomotion', 'face groom', 'other grooms', 'all other']
start = 0 
stop = 3000

first_5min_pred = predictions[start:stop] # first 5 minutes (300*10 at 10fps)
idx_end = np.hstack([np.where(np.diff(first_5min_pred) != 0)[0], len(first_5min_pred)-1])
class_id, class_count = np.unique(first_5min_pred[idx_end], return_counts=True)
print(f'{behavior_names[class_of_interest]} has {class_count[class_of_interest]} bouts within {start/10}s-->{stop/10}s')

locomotion has 50 bouts within 0.0s-->300.0s


In [ ]:
fileoutname = str.join('', (infile.rpartition('DLC')[0], '_posthoc-predictions.npy'))
np.save(fileoutname, predictions)

In [ ]:
predictions #  100ms

In [ ]:
realtime_vector = np.load('../../../raw_data/realtime_vector.npy')

In [ ]:
realtime_vector_s = realtime_vector*10

In [ ]:
realtime_vector_int = np.int64(realtime_vector_s)

In [ ]:
realtime_vector_int

In [ ]:
[predictions[i] for i in realtime_vector_int+3]

In [ ]:
len(np.where(np.array([predictions[i] for i in realtime_vector_int])==2)[0])/len([predictions[i] for i in realtime_vector_int])

In [ ]:
np.where(np.array([predictions[i] for i in realtime_vector_int])==2)[0]

In [ ]:
len([predictions[i] for i in realtime_vector_int])

In [ ]:
print(first_5min_pred[-87:])

In [ ]:
first_5min_pred = predictions[:3000] # first 5 minutes (300*10 at 10fps)
print(first_5min_pred.shape, np.diff(first_5min_pred).shape)
idx_end = np.hstack([np.where(np.diff(first_5min_pred) != 0)[0], len(first_5min_pred)-1])

# np.

In [ ]:
predictions

In [ ]:
since_40min_pred = predictions[15000:18000] # first 5 minutes (300*10 at 10fps)
print(since_40min_pred.shape, np.diff(since_40min_pred).shape)
idx_end = np.hstack([np.where(np.diff(since_40min_pred) != 0)[0], len(since_40min_pred)-1])

# np.

In [ ]:
9000/(10*60)

In [ ]:
len(idx_end)

In [ ]:
first_5min_pred[idx_end[-2:]]

In [ ]:
35/50

In [ ]:
30/38

In [ ]:
71/190

In [ ]:
np.unique(first_5min_pred[idx_end], return_counts=True)

In [ ]:
np.unique(since_40min_pred[idx_end], return_counts=True)

In [ ]:
print(predictions[:160])

In [ ]:
print(predictions[170:210])

In [ ]:
print(predictions[395:400])

In [ ]:
print(predictions[405:415])

In [ ]:
print(predictions[415:430])

In [ ]:
transition_frame = np.where(np.diff(predictions)!=0)[0]
predictions[transition_frame[0]]


## put the below code in another notebook 
#### loop through all .npy files once analyzed

In [ ]:
all_predictions = []
for _ in range(6):
    # join duplicates of the same array
    predictions = np.load(fileoutname)
    all_predictions.append(predictions)
# all_predictions
all_predictions_arr = np.hstack(all_predictions)

In [ ]:
class_of_interest=17
jitter=0.5
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.hlines(class_of_interest-jitter, 0, len(all_predictions_arr), 'r', alpha=0.7)
ax.hlines(class_of_interest+jitter, 0, len(all_predictions_arr), 'r', alpha=0.7)
ax.vlines(0-jitter, class_of_interest-jitter, class_of_interest+jitter, 'r', alpha=0.7)
ax.vlines(len(all_predictions_arr)+jitter, class_of_interest-jitter, class_of_interest+jitter, 'r', alpha=0.7)
ax.scatter(np.arange(len(all_predictions_arr)), all_predictions_arr, s=0.5, color='k', alpha=0.3)
ax.set_xticks(np.arange(0, len(all_predictions_arr), 300))
ax.set_xticklabels(np.arange(0, len(all_predictions_arr)/10, 30))
ax.set_ylabel('behavior ID')
ax.set_xlabel('time (s)')
fig.savefig('./test.png', dpi=600)

In [ ]:
all_predictions_arr.shape

In [ ]:
bin_start = np.arange(0, np.max(len(all_predictions_arr)), 600)
bin_end = np.arange(600, np.max(len(all_predictions_arr))+600, 600)
print(bin_start, bin_end)
binned_predictions = []
for b in range(len(bin_start)):
    binned_predictions.append(all_predictions_arr[bin_start[b]:bin_end[b]])


In [ ]:
class_interest_binned = [len(np.where(binned_predictions[b] == class_of_interest)[0]) for b in range(len(binned_predictions))]

plt.bar(np.arange(len(class_interest_binned)), class_interest_binned)

In [ ]:
np.arange(0, len(predictions), 300), np.arange(0, len(predictions)/10, 30)